In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import pandas as pd
import numpy as np

sample_submission = pd.read_csv('/content/drive/MyDrive/DACON_elecCarPrice_Pred/sample_submission.csv', encoding = 'utf-8')
train = pd.read_csv('/content/drive/MyDrive/DACON_elecCarPrice_Pred/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/DACON_elecCarPrice_Pred/test.csv', encoding = 'utf-8')

In [67]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        846 non-null    object 
 1   제조사       846 non-null    object 
 2   모델        846 non-null    object 
 3   차량상태      846 non

(None, None)

In [68]:
train.isnull().sum(), test.isnull().sum()

(ID             0
 제조사            0
 모델             0
 차량상태           0
 배터리용량       2711
 구동방식           0
 주행거리(km)       0
 보증기간(년)        0
 사고이력           0
 연식(년)          0
 가격(백만원)        0
 dtype: int64,
 ID            0
 제조사           0
 모델            0
 차량상태          0
 배터리용량       304
 구동방식          0
 주행거리(km)      0
 보증기간(년)       0
 사고이력          0
 연식(년)         0
 dtype: int64)

# 배터리용량 NaN 채우기
1. train, test 데이터합치기 = combined
2. combined에서 배터리용량이 NaN이 아닌 데이터만 combined2에 넣기
3. combined를 KNN 전처리를 하기 위해, combined3에 넣기
4. KNN 전처리
5. 그룹핑 전처리


In [69]:
# 데이터 합치기 전에 데이터 분리 기준 설정
div = len(train)
div

7497

In [70]:
train.shape

(7497, 11)

In [71]:
test.shape

(846, 10)

In [72]:
# 배터리용량 NaN 채우기 위해 가격 컬럼 따로 수집
target = train.pop('가격(백만원)')
target.shape

(7497,)

In [73]:
# 배터리옹량 NaN 값 찾기 위해 DF 합치기
combined = pd.concat([train, test], axis=0)
combined.shape

(8343, 10)

In [74]:
# 배터리용량 NaN 값 찾기 위해 ID 값 따로 수집
combinedID = combined.pop('ID')
combined.shape

(8343, 9)

In [75]:
combined.isnull().sum()

,0
제조사,0
모델,0
차량상태,0
배터리용량,3015
구동방식,0
주행거리(km),0
보증기간(년),0
사고이력,0
연식(년),0


In [76]:
# 배터리용량 결측치

cond1 = combined['배터리용량'].isnull()
cond1

,배터리용량
0,False
1,False
2,False
3,True
4,False
...,...
841,False
842,False
843,False
844,False


In [77]:
combined_rf = combined[~cond1]
combined_rf

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
0,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2
1,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0
2,A사,eT,Brand New,91.200,AWD,2361,7,No,0
4,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0
5,H사,ION6,Pre-Owned,58.162,AWD,103100,3,No,0
...,...,...,...,...,...,...,...,...,...
841,P사,TayGTS,Pre-Owned,64.683,AWD,117298,2,No,0
842,V사,ID4,Pre-Owned,55.547,AWD,72308,0,No,0
843,V사,ID4,Pre-Owned,55.547,AWD,124537,0,No,0
844,A사,Q4eT,Nearly New,69.646,AWD,15629,4,No,0


# KNN 대치법
1. KNN대치법은 NaN을 n개의 이웃까지의 값으로 대치

In [78]:
#knn용 데이터
combined_knn = combined
cols = combined_knn.select_dtypes(include=object).columns
cols

Index(['제조사', '모델', '차량상태', '구동방식', '사고이력'], dtype='object')

In [79]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
  combined_knn[col] = le.fit_transform(combined_knn[col])

combined_knn.head()


,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
0,4,16,1,86.077,0,13642,0,0,2
1,3,10,1,56.000,1,10199,6,0,0
2,0,17,0,91.200,0,2361,7,0,0
3,0,12,1,NaN,0,21683,3,0,0
4,1,19,2,61.018,0,178205,1,0,0


In [80]:
combined_knn['배터리용량'].value_counts()

,count
배터리용량,
90.000,625
56.000,361
46.000,250
68.488,227
76.093,209
...,...
62.230,1
54.390,1
61.140,1


In [81]:
from sklearn.impute import KNNImputer

# 예시 데이터 생성: 일부 값이 np.nan으로 결측 처리됨

print("원본 데이터프레임:")
print(combined_knn)

# KNNImputer 객체 생성 (n_neighbors 값을 데이터 특성에 맞게 조정 가능)
imputer = KNNImputer(n_neighbors=5)

# fit_transform을 통해 결측치 보완 (KNN 대치법 적용)
combined_knn = pd.DataFrame(imputer.fit_transform(combined_knn), columns=combined_knn.columns)
print("\n결측치 보완 후 데이터프레임:")
print(combined_knn)


원본 데이터프레임:
     제조사  모델  차량상태   배터리용량  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)
0      4  16     1  86.077     0     13642        0     0      2
1      3  10     1  56.000     1     10199        6     0      0
2      0  17     0  91.200     0      2361        7     0      0
3      0  12     1     NaN     0     21683        3     0      0
4      1  19     2  61.018     0    178205        1     0      0
..   ...  ..   ...     ...   ...       ...      ...   ...    ...
841    4  16     2  64.683     0    117298        2     0      0
842    6   1     2  55.547     0     72308        0     0      0
843    6   1     2  55.547     0    124537        0     0      0
844    0  11     1  69.646     0     15629        4     0      0
845    1  18     2  46.000     0     53945        0     0      0

[8343 rows x 9 columns]

결측치 보완 후 데이터프레임:
      제조사    모델  차량상태    배터리용량  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)
0     4.0  16.0   1.0  86.0770   0.0   13642.0      0.0   0.0    2.0
1     3.0  10.0   1.0  56.00

In [82]:
# train, test ID 데이터 분리
train_ID = combinedID[:div]
test_ID = combinedID[div:]
train_ID.shape, test_ID.shape

((7497,), (846,))

In [83]:
# train, test 데이터 분리
train = combined_knn[:div]
test = combined_knn[div:]
train.shape, test.shape

((7497, 9), (846, 9))

In [84]:
train.isnull().sum()

,0
제조사,0
모델,0
차량상태,0
배터리용량,0
구동방식,0
주행거리(km),0
보증기간(년),0
사고이력,0
연식(년),0


In [85]:
train.head()

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
0,4.0,16.0,1.0,86.0770,0.0,13642.0,0.0,0.0,2.0
1,3.0,10.0,1.0,56.0000,1.0,10199.0,6.0,0.0,0.0
2,0.0,17.0,0.0,91.2000,0.0,2361.0,7.0,0.0,0.0
3,0.0,12.0,1.0,61.7516,0.0,21683.0,3.0,0.0,0.0
4,1.0,19.0,2.0,61.0180,0.0,178205.0,1.0,0.0,0.0


In [86]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   제조사       7497 non-null   float64
 1   모델        7497 non-null   float64
 2   차량상태      7497 non-null   float64
 3   배터리용량     7497 non-null   float64
 4   구동방식      7497 non-null   float64
 5   주행거리(km)  7497 non-null   float64
 6   보증기간(년)   7497 non-null   float64
 7   사고이력      7497 non-null   float64
 8   연식(년)     7497 non-null   float64
dtypes: float64(9)
memory usage: 527.3 KB


In [87]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 7497 to 8342
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   제조사       846 non-null    float64
 1   모델        846 non-null    float64
 2   차량상태      846 non-null    float64
 3   배터리용량     846 non-null    float64
 4   구동방식      846 non-null    float64
 5   주행거리(km)  846 non-null    float64
 6   보증기간(년)   846 non-null    float64
 7   사고이력      846 non-null    float64
 8   연식(년)     846 non-null    float64
dtypes: float64(9)
memory usage: 59.6 KB


In [88]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_tr.shape,X_val.shape,y_tr.shape,y_val.shape)

(5997, 9) (1500, 9) (5997,) (1500,)


In [95]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error as rmse
rf = RandomForestRegressor(random_state=0, n_estimators=100, max_depth=9)
rf.fit(X_tr,y_tr)
pred_rf = rf.predict(X_val)
rmse(y_val, pred_rf)

1.35640975970377

In [90]:
# test예측 및 csv 생성
rf_pred = rf.predict(test)
submit = pd.DataFrame({
    'ID': test_ID,
    'pred': rf_pred
}).to_csv('result.csv',index=False)

In [91]:
result = pd.read_csv('result.csv')
result.head()

,ID,pred
0,TEST_000,130.717691
1,TEST_001,80.093835
2,TEST_002,64.703515
3,TEST_003,34.501844
4,TEST_004,48.135938


# NaN 그룹핑 대치